In [1]:
from skyfield.api import N, W, wgs84, load
from skyfield.functions import length_of
import folium
import requests
from geopy.geocoders import Nominatim
from IPython.display import display

In [51]:
# Function to get lat/long using Nominatim API
def get_city_coordinates(city_name):
    geolocator = Nominatim(user_agent="city_locator")
    location = geolocator.geocode(city_name)
    if location:
        return location.latitude, location.longitude
    else:
        print(f"City {city_name} not found.")
        return None

# Function to calculate the distance from the Earth's core
def get_distance_to_core(lat, lon, elevation):
    ts = load.timescale()
    t = ts.utc(2023, 1, 1)
    city_location = wgs84.latlon(lat * N, lon * W, elevation_m=elevation)
    distance = length_of(city_location.at(t).position.m)  # Distance in meters
    return distance / 1000  # Convert to kilometers

# Function to add a marker and distance to the map with color coding based on distance
def add_city_to_map(city_name, lat, lon, elevation, distance, map_object):
    # Assign color based on distance to core
    if distance <= 6367:  # Closest (Red)
        color = "orange"
    elif distance <= 6369:  # Medium-Close (orange)
        color = "cadetblue"
    elif distance <= 6370:  # Medium-Close (Green)
        color = "lightblue"
    else:  # Furthest (Blue)
        color = "lightgray"
    
    folium.Marker(
        location=[lat, lon],
        popup=f"{city_name}: {distance:.2f} km to Earth's core",
        icon=folium.Icon(color=color, prefix='fa',icon='flag')
    ).add_to(map_object)

# Create a folium map
city_map = folium.Map(location=[20, 0], zoom_start=2, tiles=None)  # Set tiles to None to add custom basemap
'''Esri_WorldImagery = folium.TileLayer(
    'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community'
).add_to(city_map)'''

CartoDB_DarkMatter = folium.TileLayer(
    'https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png',attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors &copy; <a href="https://carto.com/attributions">CARTO</a>').add_to(city_map);




# Example city names (you can edit or extend this list, elevation must be provided)
cities = [
    {"name": "Seattle", "elevation": 609},  # Elevation in meters
    {"name": "New York", "elevation": 10},
    {"name": "Paris", "elevation": 35},
    {"name": "Rascafria", "elevation": 1163},
    {"name": "Las Vegas", "elevation": 610},
    {"name": "Vancouver", "elevation": 2},
    {"name": "San Diego", "elevation": 22},
    {"name": "Stowe", "elevation": 271},
    {"name": "London", "elevation": 24},
    {"name": "Los Cabos", "elevation": 20},
    {"name": "Rascafria", "elevation": 1163},
    {"name": "Bar Harbor", "elevation": 20},
    {"name": "Cádiz", "elevation": 11},
    {"name": "Lucerne", "elevation": 435}
]

# List to store city data with distance
city_data = []

# Iterate over cities to fetch coordinates and calculate distances
for city in cities:
    city_name = city["name"]
    elevation = city["elevation"]
    coordinates = get_city_coordinates(city_name)
    if coordinates:
        lat, lon = coordinates
        distance = get_distance_to_core(lat, lon, elevation)
        city_data.append({"name": city_name, "lat": lat, "lon": lon, "elevation": elevation, "distance": distance})

# Sort cities by distance to Earth's core (ascending order)
sorted_city_data = sorted(city_data, key=lambda x: x["distance"])

# Add sorted cities to the map with color coding
for city in sorted_city_data:
    add_city_to_map(city["name"], city["lat"], city["lon"], city["elevation"], city["distance"], city_map)


#Create html legend for lower left corner 
legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: auto; 
                background-color: #212122; color: #fff;  
                z-index: 9999; font-size: 14px; padding: 15px 10px;">
        <b style="margin-bottom: 10px; display: block;">Distance to Earth's Core</b>
        <div style="margin-bottom: 8px;">
            <i style="height: 8px; vertical-align: -10px; background-color: orange; padding: 5px 10px; color: #fff; display: inline-block;">&nbsp;&nbsp;</i> Closest
        </div>
        <div style="margin-bottom: 8px;">
            <i style="height: 8px; vertical-align: -10px;background-color: cadetblue; padding: 5px 10px; display: inline-block;">&nbsp;&nbsp;</i> Medium
        </div>
        <div style="margin-bottom: 8px;">
            <i style="height: 8px; vertical-align: -10px;background-color: lightblue; padding: 5px 10px; display: inline-block;">&nbsp;&nbsp;</i> Medium
        </div>
        <div style="margin-bottom: 0;">
            <i style="height: 8px; vertical-align: -10px;background-color: lightgray; padding: 5px 10px; color: #fff; display: inline-block;">&nbsp;&nbsp;</i> Farthest
        </div>
    </div>
'''

# Add the legend to the map
city_map.get_root().html.add_child(folium.Element(legend_html))

# Display the map inline in Jupyter Notebook
display(city_map)


In [53]:
city_map.save("cities_map.html")
print("Map saved as cities_map.html")

Map saved as cities_map.html
